In [1]:
import pandas as pd
import numpy as np

In [2]:
path=""
X =pd.read_csv(path+"Xtr.csv", header=None)
Y =pd.read_csv(path+"Ytr.csv")

In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(
        X, Y["Prediction"], test_size=0.2)

In [4]:
X.shape


(5000, 784)

In [5]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
from sklearn.metrics import accuracy_score
predicted_labels = svc.predict(X_test)
print accuracy_score(predicted_labels,y_test)


0.847


In [7]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(kernel = 'poly', degree = 3, n_components= 35)
kpca.fit(X_train)
X_train_kpca = kpca.transform(X_train)
X_test_kpca = kpca.transform(X_test)

In [8]:
X_test.shape, X_train_kpca.shape

((1000, 784), (4000, 35))

In [10]:
%%time 
from sklearn.grid_search import GridSearchCV 
from sklearn.neighbors import KNeighborsClassifier
NN = {'n_neighbors': range(1,15,1)}
knn = KNeighborsClassifier(weights='distance')
grid_search = GridSearchCV(knn, NN, n_jobs = 3)
alg = grid_search.fit(X_train_kpca, y_train)
#alg = knn.fit(X_train_kpca, y_train)
predicted_label = alg.predict(X_test_kpca)

print("SVM - Best C & associated score", grid_search.best_params_, grid_search.best_score_)
print("SVM - Score on test_data : ", accuracy_score(y_train, alg.predict(X_train_kpca)))
print("SVM - Score on test_data : ", accuracy_score(y_test, predicted_label))

('SVM - Best C & associated score', {'n_neighbors': 4}, 0.89500000000000002)
('SVM - Score on test_data : ', 1.0)
('SVM - Score on test_data : ', 0.91300000000000003)
CPU times: user 3.63 s, sys: 67.4 ms, total: 3.7 s
Wall time: 17 s


In [7]:
from sklearn.grid_search import GridSearchCV 
Cs = {'C': np.logspace(-5, 0, 10)}

grid_search = GridSearchCV(SVC(cache_size= 1000), Cs, n_jobs = 3)
alg = grid_search.fit(X_train, y_train)
predicted_label = alg.predict(X_test)

print("SVM - Best C & associated score", grid_search.best_params_, grid_search.best_score_)
print("SVM - Score on test_data : ", accuracy_score(y_train, alg.predict(X_train)))
print("SVM - Score on test_data : ", accuracy_score(y_test, predicted_label))


('SVM - Best C & associated score', {'C': 1.0}, 0.82999999999999996)
('SVM - Score on test_data : ', 0.86275000000000002)
('SVM - Score on test_data : ', 0.85999999999999999)


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
svc = SVC(kernel='poly')
svc.fit(X_train,y_train)
predicted_labels = svc.predict(X_test)
print accuracy_score(predicted_labels,y_test)


0.12


In [9]:
svc = SVC(kernel='linear')
svc.fit(X_train,y_train)
predicted_labels = svc.predict(X_test)
print accuracy_score(predicted_labels,y_test)


0.852
